# Module 4: The B-Tree
**Goal**: Shatter the illusion that Indexes are "Magic Fast Buttons."
Most developers know that adding an index makes queries fast, but they don't know why. It isn't magic; it is a specific data structure that creates a tradeoff: you trade write speed (maintaining order) for read speed (finding data quickly).

In this module, we will explore:
1. **The Algorithm**: Why sorting data transforms a search from "checking every page" to "flipping a few pages."
2. **The Physics**: Observing Postgres skipping thousands of disk pages using an Index Scan.
3. **The Clustered Index**: How physical disk sorting impacts range queries.

---

## 0. Setup: The Laboratory
We will use **Postgres** for this module, as it provides excellent tooling (EXPLAIN ANALYZE) to see exactly how the database engine is reading data from the disk.

We will prepare two tables using our users dataset:
1. `users_heap`: A standard table with no index.
2. `users_btree`: The same data, but with a B-Tree Index on `user_id`.

In [ ]:
import pandas as pd
import psycopg2
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Database Connection
conn = psycopg2.connect(
    host="db_int_opt",
    user="admin",
    password="password",
    dbname="db_int_opt"
)
conn.autocommit = True
cur = conn.cursor()

# Visualization Settings
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

print("✅ Laboratory Environment Ready.")

# --- DATA PREPARATION ---
print("⚙️ Loading Data into Postgres...")

# Load CSV
df = pd.read_csv('../data/users.csv')

if 'status' in df.columns:
    df.rename(columns={'status': 'account_status'}, inplace=True)

df_clean = df[['user_id', 'account_status', 'email']]

# Create tables
cur.execute("DROP TABLE IF EXISTS users_heap;")
cur.execute("DROP TABLE IF EXISTS users_btree;")

# Table 1: No Index (The Heap)
cur.execute("""
    CREATE TABLE users_heap (
        user_id INT,
        account_status TEXT,
        email TEXT
    );
""")

# Table 2: With Index (The B-Tree)
cur.execute("""
    CREATE TABLE users_btree (
        user_id INT,
        account_status TEXT,
        email TEXT
    );
""")

# Bulk Insert (Using COPY for speed)
from io import StringIO
buffer = StringIO()
df_clean.to_csv(buffer, index=False, header=False)
buffer.seek(0)

# Load into both
cur.copy_expert("COPY users_heap FROM STDIN WITH CSV", buffer)
buffer.seek(0)
cur.copy_expert("COPY users_btree FROM STDIN WITH CSV", buffer)

# CREATE THE INDEX on the second table
print("⚙️ Building B-Tree Index...")
cur.execute("CREATE INDEX idx_user_id ON users_btree(user_id);")
# Force Postgres to gather statistics immediately so our experiments are accurate
cur.execute("ANALYZE users_heap;")
cur.execute("ANALYZE users_btree;")

print("✅ Data Loaded: 100k Rows in 'users_heap' and 'users_btree'.")

----

## 4.1 The Sorted Structure (The Phone Book)
Before we look at the database, we must understand the math.

Imagine I ask you to find the name "Zoltan" in a phone book.
- **Scenario A (Random)**: The names are printed in random order. You must read every single name from the start until you find "Zoltan".
- **Scenario B (Sorted)**: The names are alphabetical. You open the book to the middle. If you see "M", you know "Z" is in the right half. You split the remaining pages in half again.

This acts as a "filter." Every time you make a decision, you discard **half** of the remaining data.

### Experiment 4.1: Linear Scan vs. Binary Search
We will simulate this in Python memory. We will search for a number in a list of 100,000 integers.

**Hypothesis**: How much faster is Binary Search? 2x? 10x?

In [ ]:
import bisect 
import random

# 1. Prepare Data
ids_list = df['user_id'].tolist()
target_id = ids_list[-1] # Let's look for the very last item (Worst case for linear)

# 2. Linear Search (The "Scan")
start_linear = time.time()
found = False
for i in ids_list:
    if i == target_id:
        found = True
        break
end_linear = time.time()

# 3. Binary Search (The "Seek")
# We must sort the list first to use binary search
ids_sorted = sorted(ids_list) 

start_binary = time.time()
# bisect is Python's built-in binary search algorithm
index = bisect.bisect_left(ids_sorted, target_id)
end_binary = time.time()

# 4. Results
time_linear = (end_linear - start_linear) * 1000 # to ms
time_binary = (end_binary - start_binary) * 1000 # to ms

print(f"Linear Search Time: {time_linear:.4f} ms")
print(f"Binary Search Time: {time_binary:.4f} ms")
print(f"Speedup Factor: {time_linear / time_binary:.0f}x")

# Visualize
plt.bar(['Linear Scan (Unsorted)', 'Binary Search (Sorted)'], [time_linear, time_binary], color=['red', 'green'])
plt.title("Search Algorithm Efficiency (Lower is Better)")
plt.ylabel("Time (milliseconds)")
plt.yscale('log') # Log scale because the difference is massive
plt.show()

## Observation
You likely see a massive difference. Linear search performance degrades linearly ($O(n)$) as data grows. Binary search is Logarithmic ($O(\log n)$). Even if we had 1 billion rows, the binary search would only take about 30 steps.

---

## 4.2 Logarithmic Time ($O(\log n)$) in the Database
Now, let's see if this physics applies to the actual Postgres database engine.

We will query for a specific `user_id`.
- `users_heap`: Postgres must scan the whole table (Sequential Scan).
- `users_btree`: Postgres will traverse the B-Tree index to find the pointer to the specific row.

### Experiment 4.2: Sequential Scan vs. Index Scan

We will use the SQL command `EXPLAIN (ANALYZE, BUFFERS)` to ask Postgres exactly how much work it did. We are looking for "Buffers"—this is the number of 8KB pages Postgres had to read from memory/disk.

In [ ]:
def measure_query_cost(table_name, search_id):
    query = f"EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON) SELECT * FROM {table_name} WHERE user_id = {search_id};"
    cur.execute(query)
    plan = cur.fetchone()[0][0]
    
    cost = plan['Plan']['Total Cost']
    execution_time = plan['Execution Time']
    # Extract buffer hits (shared blocks hit)
    buffers = plan['Plan'].get('Shared Hit Blocks', 0) + plan['Plan'].get('Shared Read Blocks', 0)
    
    return execution_time, buffers

# Search for a random ID
search_target = 99999

time_heap, buffers_heap = measure_query_cost("users_heap", search_target)
time_btree, buffers_btree = measure_query_cost("users_btree", search_target)

print(f"HEAP TABLE: {time_heap:.3f} ms | Pages Read: {buffers_heap}")
print(f"BTREE TABLE: {time_btree:.3f} ms | Pages Read: {buffers_btree}")

# Visualize Pages Read (The Physical Work)
plt.figure(figsize=(8, 5))
plt.bar(['Heap (Seq Scan)', 'B-Tree (Index Scan)'], [buffers_heap, buffers_btree], color=['#e74c3c', '#2ecc71'])
plt.title("Physical I/O Cost: Pages Read from Memory/Disk")
plt.ylabel("Number of 8KB Pages")
for i, v in enumerate([buffers_heap, buffers_btree]):
    plt.text(i, v + 1, str(v), ha='center', fontweight='bold')
plt.show()

### The Physics: Why so few pages?
The B-Tree is a short, wide tree.
1. **Root Node**: Postgres reads the top page. It says "IDs 0-50k go Left, 50k-100k go Right."
2. **Branch Nodes**: It follows the pointer down.
3. **Leaf Node**: It hits the bottom, finds the exact location of the data on the disk (Tuple ID).
4. **The Table**: It jumps to the specific Heap page to grab the row.

This usually takes 3 to 4 "hops" (Page reads), regardless of whether the table has 100k rows or 10M rows. The Heap Scan, conversely, had to read every single page hoping to find the ID.

---

## 4.3 The Clustered Index (Physical Sorting)
An index helps you find a single needle. But what if you want a range of needles?

If your data is physically scattered on the disk (Random), fetching a range means jumping around frantically (Random I/O). If your data is physically sorted (Clustered), fetching a range is a smooth sequential read.

**Note**: Postgres does not automatically maintain Clustered Indexes (unlike SQL Server or MySQL InnoDB), but we can simulate it by loading data in sorted order vs. shuffled order.

### Experiment 4.3: Range Query on Sorted vs. Shuffled Data
We will load `orders_sorted.csv` and `orders_shuffled.csv`. Both contain the same data. We will query a date range.

**Hypothesis**: The sorted table will be faster because the data for "January 2023" resides on adjacent pages.

In [ ]:
# --- 2. DEFINE SCHEMA & TABLES ---
cur.execute("DROP TABLE IF EXISTS orders_sorted;")
cur.execute("DROP TABLE IF EXISTS orders_shuffled;")

cur.execute("""
    CREATE TABLE orders_sorted (
        order_id INT,
        user_id INT,
        order_date TIMESTAMP,
        amount FLOAT
    );
""")

cur.execute("""
    CREATE TABLE orders_shuffled (
        order_id INT,
        user_id INT,
        order_date TIMESTAMP,
        amount FLOAT
    );
""")

# --- 3. ROBUST DATA LOADING (CLEANING) ---
def clean_and_load(file_path, table_name):
    print(f"   Processing {file_path}...")
    df = pd.read_csv(file_path)
    
    # Handle missing 'amount' column by faking it or using quantity
    if 'amount' not in df.columns:
        if 'quantity' in df.columns:
             df['amount'] = df['quantity'] * 10.0 
        else:
             df['amount'] = 0.0

    # FORCE column selection and order to match SQL exactly
    # This prevents the "BadCopyFileFormat" error
    df_clean = df[['order_id', 'user_id', 'order_date', 'amount']]
    
    # Stream to DB
    buffer = StringIO()
    df_clean.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV", buffer)

clean_and_load('../data/orders_sorted.csv', 'orders_sorted')
clean_and_load('../data/orders_shuffled.csv', 'orders_shuffled')

# --- 4. BUILD INDEXES ---
print("⚙️ Building Indexes (The 'Phone Book')...")
cur.execute("CREATE INDEX idx_date_sorted ON orders_sorted(order_date);")
cur.execute("CREATE INDEX idx_date_shuffled ON orders_shuffled(order_date);")
cur.execute("ANALYZE orders_sorted; ANALYZE orders_shuffled;")

# --- 5. DYNAMIC RANGE FINDING ---
# Find a date that actually exists in the data
cur.execute("SELECT order_date FROM orders_sorted ORDER BY order_date LIMIT 1 OFFSET 25000;")
start_date_raw = cur.fetchone()[0]

# Ensure it's a timestamp object
if isinstance(start_date_raw, str):
    start_date_obj = pd.to_datetime(start_date_raw)
else:
    start_date_obj = start_date_raw

# Define a NARROW range (1 Day) to encourage Index usage
end_date_obj = start_date_obj + pd.DateOffset(days=1)

start_str = start_date_obj.strftime('%Y-%m-%d %H:%M:%S')
end_str = end_date_obj.strftime('%Y-%m-%d %H:%M:%S')

print(f"🎯 Targeted Range: {start_str} to {end_str}")

# --- 6. EXECUTE EXPERIMENT (FORCED INDEX) ---
# We disable Sequential Scan to force the database to use the index,
# revealing the true cost of "hopping" around the disk.
cur.execute("SET enable_seqscan = OFF;")

query_template = f"""
    EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON) 
    SELECT * FROM {{}} 
    WHERE order_date BETWEEN '{start_str}' AND '{end_str}';
"""

def measure_cost(table):
    cur.execute(query_template.format(table))
    plan = cur.fetchone()[0][0]
    buffers = plan['Plan']['Shared Hit Blocks'] + plan['Plan']['Shared Read Blocks']
    return plan['Execution Time'], buffers

print("\n🚀 Running Physics Experiment...")
t_sorted, b_sorted = measure_cost("orders_sorted")
t_shuffled, b_shuffled = measure_cost("orders_shuffled")

# Reset config
cur.execute("SET enable_seqscan = ON;")

print(f"Sorted Table:   {t_sorted:.3f} ms | Pages Read: {b_sorted}")
print(f"Shuffled Table: {t_shuffled:.3f} ms | Pages Read: {b_shuffled}")

# --- 7. VISUALIZATION ---
fig, ax = plt.subplots(figsize=(8,5)) 

# Plot
bars = ax.bar(['Physically Sorted', 'Physically Shuffled'], [t_sorted, t_shuffled], color=['#3498db', '#e67e22'])

# Styling
ax.set_ylabel('Execution Time (ms)')
ax.set_title(f'The "Clustering Factor": Impact of Physical Order', pad=20)
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add Data Labels
for bar, pages in zip(bars, [b_sorted, b_shuffled]):
    height = bar.get_height()
    label_height = height if height > 0 else 0.1 # prevention for 0 height
    ax.text(bar.get_x() + bar.get_width()/2., label_height,
            f'{height:.1f} ms\n({pages} Pages)',
            ha='center', va='bottom', fontweight='bold')


plt.show()

### Conclusion: Correlation
In the **Sorted** table, the index points to rows that are sitting next to each other on the disk. The database reads one block and gets 50 relevant rows. In the **Shuffled** table, the index points to rows scattered everywhere. The database reads a block, finds 1 row, and has to jump to a new block for the next row.

Take a second to really appreciate the difference in pages between the two searches. We know now that a Postgres page is 8 KB, and the shuffled search required loading over 3,000 more pages. That equates to over a 24 MB increase in I/O for the shuffled data.

This is why **Clustered Indexes** (or partitioning by time) are critical for time-series data.